In [4]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

# =========================
# LOAD DATA
# =========================
df = pd.read_csv("Play Store Data.csv")

# =========================
# CLEAN REQUIRED COLUMNS
# =========================

df["Installs"] = df["Installs"].astype(str).str.replace("+","").str.replace(",","")
df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")

df["Price"] = df["Price"].astype(str).str.replace("$","")
df["Price"] = pd.to_numeric(df["Price"], errors="coerce").fillna(0)

df["Size_MB"] = df["Size"].astype(str).str.replace("M","")
df["Size_MB"] = pd.to_numeric(df["Size_MB"], errors="coerce")

df["Android_Num"] = df["Android Ver"].str.extract(r'(\d+\.\d+)')
df["Android_Num"] = pd.to_numeric(df["Android_Num"], errors="coerce")

df["Revenue"] = df["Installs"] * df["Price"]

# =========================
# APPLY FILTERS
# =========================

filtered = df[
    (df["Installs"] >= 10000) &
    (df["Revenue"] >= 10000) &
    (df["Android_Num"] > 4.0) &
    (df["Size_MB"] > 15) &
    (df["Content Rating"] == "Everyone") &
    (df["App"].str.len() <= 30)
]

# =========================
# TOP 3 CATEGORIES
# =========================

top3 = (
    filtered.groupby("Category")["Installs"]
    .sum()
    .nlargest(3)
    .index
)

top_df = filtered[filtered["Category"].isin(top3)]

# =========================
# SUMMARY FREE VS PAID
# =========================

summary = (
    top_df.groupby("Type")
    .agg(
        Avg_Installs=("Installs","mean"),
        Total_Revenue=("Revenue","sum")
    )
    .reset_index()
)

# =========================
# TIME CHECK — 1PM–2PM IST
# =========================

hour = datetime.now().hour

if 13 <= hour < 14:

    fig = go.Figure()

    fig.add_bar(
        x=summary["Type"],
        y=summary["Avg_Installs"],
        name="Average Installs"
    )

    fig.add_trace(go.Scatter(
        x=summary["Type"],
        y=summary["Total_Revenue"],
        name="Revenue",
        yaxis="y2",
        mode="lines+markers"
    ))

    fig.update_layout(
        title="Free vs Paid — Avg Installs vs Revenue (Top 3 Categories)",
        yaxis=dict(title="Average Installs"),
        yaxis2=dict(
            title="Revenue",
            overlaying="y",
            side="right"
        )
    )

    fig.show()

else:
    print("Graph hidden — visible only between 1 PM and 2 PM IST")


Graph hidden — visible only between 1 PM and 2 PM IST
